In [1]:
import pandas as pd
import numpy as np
import regex as re

import pickle

# from random import choice
from useful_functions import *

### Merging everthing one last time:

All related variables from this nb are in 8qdfr variables folder!

In [2]:
# date_dfs:
q1_date_df = pickle_loader("q1_date_df")
qdfr_run_succ = pickle_loader("qdfr_run_succ")


In [3]:
q1_date_df = pd.concat([q1_date_df,qdfr_run_succ.drop(["country_list","time_delta"], axis=1)], verify_integrity=True)

In [5]:
# rem_dfs:
q1_rem_df = pickle_loader("q1_remainder_df")
qdfr_run_rem = pickle_loader("qdfr_run_rem")


In [6]:
q1_rem_df = pd.concat([q1_rem_df,qdfr_run_rem.set_index("DOI")])

In [7]:
# author_dfs:
q1_author_df = pickle_loader("q1_author_df")
qdfr_author_df = pickle_loader("qdfr_author_df")


In [8]:
q1_author_df = pd.concat([q1_author_df,qdfr_author_df],ignore_index=True, verify_integrity=True)

In [9]:
# first_author_dfs:
q1_first_author_df = pickle_loader("q1_first_author_df")
qdfr_first_author_df = pickle_loader("qdfr_first_author_df")

In [22]:
qdfr_first_author_df = qdfr_first_author_df[~qdfr_first_author_df.index.duplicated()]

In [23]:
q1_first_author_df = pd.concat([q1_first_author_df.drop(["country_count","most_common_country"], axis=1),qdfr_first_author_df], verify_integrity=True)

Everything is merged & ready to be dumped

In [25]:
pickle_dumper(q1_date_df, "q1_date_df")
pickle_dumper(q1_rem_df, "q1_remainder_df")
pickle_dumper(q1_author_df, "q1_author_df")
pickle_dumper(q1_first_author_df, "q1_first_author_df")


After merge analysis:

In [2]:
q1_date_df = pickle_loader("q1_date_df")
q1_first_author_df = pickle_loader("q1_first_author_df")
q1_jrnl_df = pickle_loader("q1_jrnl_df")

In [5]:
with open("7b_variables/q1_a_ss", "rb") as fp:
    q1_a_ss = pickle.load(fp)

In [11]:
q1_jrnl_df.drop(["Required_Count","Required_Count_Sum","New_Additions","Current_Count"], axis=1, inplace=True)

In [15]:
q1_date_df["Published_Year"] = q1_date_df.Published_Date.map(lambda x: x[0])

In [17]:
current_pivot = pd.pivot_table(data=q1_date_df, columns="Published_Year", index="Chosen_SA", values="title", aggfunc="count")

In [34]:
q1_date_df["JRNL_SA"] = q1_date_df.JRNL_ID.map(q1_jrnl_df.SA_list.to_dict())

In [36]:
missing_pivot = (q1_a_ss - current_pivot).applymap(lambda x: x if x>0 else 0)

In [37]:
missing_pivot

Year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Sbj_Area,,,,,,,,,,,
Agricultural and Biological Sciences,0,0,0,0,0,0,0,0,0,0,0
Arts and Humanities,0,0,0,0,0,0,0,0,0,0,0
"Biochemistry, Genetics and Molecular Biology",0,0,0,0,0,0,0,0,0,0,0
"Business, Management and Accounting",0,0,0,0,0,0,0,0,0,0,0
Chemical Engineering,0,0,0,0,0,0,0,0,0,0,0
Chemistry,0,0,0,0,0,0,0,0,0,0,0
Computer Science,0,0,0,0,0,0,0,0,0,0,0
Decision Sciences,0,0,0,0,0,0,0,0,0,0,0
Dentistry,0,0,0,0,0,0,0,0,0,0,0


It is seen that there are still some "missing" articles. However, this is actually due to S_A distribution. 

At this point there are more than enough articles from jrnls to cover all cells of the pivot.

Will create a new loop to re-map SA's to cover the missing ones:

In [50]:
# Distributing extra  articles to journals:

multip = 1.5

for s_a, row in missing_pivot.iterrows(): #iterate over rows
    print(s_a)
    for year, value in row.items():
        print(year)
        if value == 0:
            pass
        else:
            try:
                sampl_indexes = q1_date_df[(q1_date_df.JRNL_SA.map(lambda x: s_a in x)) & (q1_date_df.Chosen_SA != s_a) & (q1_date_df.Published_Year == year)].sample(value).index.to_list()
            except:
                sampl_indexes = q1_date_df[(q1_date_df.JRNL_SA.map(lambda x: s_a in x)) & (q1_date_df.Chosen_SA != s_a) & (q1_date_df.Published_Year == year)].sample(frac=1).index.to_list()

            q1_date_df.loc[sampl_indexes,"Chosen_SA"] = s_a


Agricultural and Biological Sciences
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Arts and Humanities
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Biochemistry, Genetics and Molecular Biology
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Business, Management and Accounting
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Chemical Engineering
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Chemistry
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Computer Science
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Decision Sciences
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Dentistry
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Earth and Planetary Sciences
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Economics, Econometrics and Finance
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Energy
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Engineering
2010
2011
2012
2013
2014
2015
2016
2017
20

In [51]:
after_pivot = pd.pivot_table(data=q1_date_df, columns="Published_Year", index="Chosen_SA", values="title", aggfunc="count")

In [54]:
(q1_a_ss - after_pivot).applymap(lambda x: x if x>0 else 0)

Year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Sbj_Area,,,,,,,,,,,
Agricultural and Biological Sciences,0,0,0,0,0,0,0,0,0,0,0
Arts and Humanities,0,0,0,0,0,0,0,0,0,0,0
"Biochemistry, Genetics and Molecular Biology",0,0,0,0,0,0,0,0,0,0,0
"Business, Management and Accounting",0,0,0,0,0,0,0,0,0,1,0
Chemical Engineering,0,0,0,0,0,0,0,0,0,0,0
Chemistry,0,0,0,0,0,0,0,0,0,0,0
Computer Science,0,0,0,0,0,0,0,0,0,0,0
Decision Sciences,0,0,0,0,0,0,0,0,0,2,0
Dentistry,0,0,0,0,0,0,0,0,0,0,0


In [57]:
q1_jrnl_df["Current_Count"] = q1_date_df.groupby("JRNL_ID")["title"].count()

In [62]:
pickle_dumper(q1_date_df, "q1_date_df")

In [66]:
pickle_dumper(q1_jrnl_df, "q1_jrnl_df")